# Data wrangling and datetimes

Dates and times are a special kind of data type. In this notebook, we will keep exploring the `orders` and `orderlines` datasets from Eniac and learn to deal with them. 

In [ ]:
import pandas as pd

url = 'https://drive.google.com/file/d/14Y7g5ITyf6LMyPoKc9wr010V9StaCUux/view?usp=sharing' # orderlines.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)

df.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38


## Data exploration

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


Two variables need to be modified: 

* `unit_price`: it is detected as an object but it has to be a float. Why is that happening? 
* `date`: needs to be transformed to a date-time format. 

For the moment we will only focus on the `date`. Since it contains both the date and the time, we will transform the data type using the pandas method `.to_datetime()`: 

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


In [ ]:
df.describe()

You can count the quantity of observations with `.value_counts()`.

In [ ]:
df['sku'].value_counts()

**Exercise**: check whether the dates from the `orderlines` dataset match with the dates on the `orders` dataset.

In [ ]:
url = 'https://drive.google.com/file/d/14Y7g5ITyf6LMyPoKc9wr010V9StaCUux/view?usp=sharing' # orderlines.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
order_lines = pd.read_csv(path)

url = 'https://drive.google.com/file/d/1BLEHcP-9fm9Rv7A01H3co2XBMnSr66YC/view?usp=sharing' # orders.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
orders = pd.read_csv(path)

In [ ]:
orders['created_date'] = pd.to_datetime(orders['created_date'])
orders

In [ ]:
order_lines['date'] = pd.to_datetime(order_lines['date'])
order_lines

In [ ]:
#order_lines.compare(orders)

In [ ]:
df.shape

(293983, 7)

## Aggregating with pandas

* Grouping and aggregating is one of the main ways to explore data. The main tools to do that with pandas are:
    * [`pandas.DataFrame.groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html).
    * [`pandas.DataFrame.agg()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html?highlight=agg#pandas.DataFrame.agg) 
        * [`pandas.DataFrame.count()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html)

In [ ]:
df.head()

How many products do we have on each sku?

In [ ]:
df.groupby(['sku']).agg({'product_quantity':'count'})

Different levels of aggregation answer different business questions:

- **count**: How many times has each product (identified by its `sku`) been purchased in an order? 

- **sum**: How many units of each product have been sold overall? 

- **mean**: How many units/order are sold for each product?

In [ ]:
df.groupby(['sku']).agg({'product_quantity':['count', 'sum', 'mean']})

`groupby` can also be combined with other pandas functions to analyse in more depth the datasets.

In [ ]:
df.groupby('sku')[['sku','product_quantity']].describe()

Now let's see how can we combine `.groupby()` and `.aggragate()` with `.sort_values()`.

I would like to see the top 10 most sold products on our data (total product quantity). 

In [ ]:
#df.groupby(['sku'])['product_quantity'].sum().sort_values(ascending=False).head(10)
(
df
    .groupby('sku')['product_quantity']
    .sum()
    .sort_values(ascending=False).head(10)
)

How can we sort values with multiple aggregated functions?

In [ ]:
(
df
    .groupby('sku')
    .agg({'product_quantity':['sum','count','std','mean','median']})
    .sort_values(('product_quantity','mean'), ascending=False)
)

## Working with date time

We will start by creating a weekday column. You will have to combine two functions, [`pandas.DataFrame.assign()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html) from `pandas` and [`datetime.strftime()`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) from `datetime`.

First, let's talk about `.assign()`. It is a useful method for creating new columns:

In [ ]:
df.assign(new_colum = 'hi! I am a new column!').head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,new_colum
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,hi! I am a new column!
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,hi! I am a new column!
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,hi! I am a new column!
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,hi! I am a new column!
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,hi! I am a new column!


A new column often is the result of an operation between other columns in the dataframe:

In [ ]:
df.assign(total_price = df['product_quantity'] * df['unit_price']).head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74


In Pandas, we can extract strings containing elements like the day of the month, the day of the week, the hour of the day... from `datetime` objects. We do so by using `dt.strftime()` in combination with `strftime` syntax. Find the cheat sheet for this syntax [here](https://strftime.org/). 

You'll understand it better with an example. Let's create the weekday column:

In [ ]:
df.assign(week_day = df['date'].dt.strftime('%A')).head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,week_day
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,Sunday
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,Sunday
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,Sunday
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,Sunday
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,Sunday


In [ ]:
df['date'].dt.strftime('%B')

0         January
1         January
2         January
3         January
4         January
           ...   
293978      March
293979      March
293980      March
293981      March
293982      March
Name: date, Length: 293983, dtype: object

Now we can sort weekdays by amount of sales:

In [ ]:
(
df
    .assign(week_day = df['date'].dt.strftime('%A'))
    .groupby('week_day')
    .agg({'product_quantity':['sum','count']})
    .sort_values(('product_quantity','sum'), ascending=False)
)

product_quantity       
                       sum  count
week_day                         
Monday               57624  50307
Wednesday            54131  47550
Tuesday              50871  44498
Friday               49566  44027
Thursday             48431  43634
Sunday               35667  32857
Saturday             33302  31110

Let's find out the top 1 product (in product quantity) by sku for each weekday:

In [ ]:
(
df
    .assign(week_day = df['date'].dt.strftime('%A'))
    .groupby(['week_day','sku'])
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
    .reset_index()
    .groupby('week_day')
    .head(1)
)

,week_day,sku,product_quantity
0,Friday,APP1190,1827
1,Thursday,MIC0036,1389
3,Tuesday,MIC0036,1213
6,Wednesday,SEV0028,1014
10,Monday,APP0663,840
14,Saturday,MIC0036,806
23,Sunday,APP1190,692


In [ ]:
df.date

When using `pd.to_datetime` to convert "objects" like "28-07-1990 03:30:15" to a proper data type, you have captured both the date and the time. Sometimes, you might only need the date (without the time). An easy way to do this is with `dt.date`:

In [ ]:
df.date.dt.date.head(3)

0    2017-01-01
1    2017-01-01
2    2017-01-01
Name: date, dtype: object

Note that these are not actually strings, even thoguht the `dtype` that pandas displays for the whole column is `object`:

In [ ]:
df.date.dt.date[0]

datetime.date(2017, 1, 1)

In [ ]:
type(df.date.dt.date[0])

datetime.date

You can create these `datetime.date` objects from strings using the module `datetime`. This module has a handful of useful functions for dates and times, [check its documentation here](https://docs.python.org/3/library/datetime.html#date-objects).

In [ ]:
import datetime

datetime.date.fromisoformat("1990-07-28")

datetime.date(1995, 7, 28)

What we've outlined here is going to become useful and relevant whenever filtering a DataFrame by date, as you will see in the following challenges:

# CHALLENGES

**1. Which are the top 6 orders with the highest number of products sold?**

In [ ]:
df.groupby(['id_order'])['product_quantity'].count().sort_values(ascending=False).head(6)

id_order
395611    256
484334    140
301934    131
329309     73
349364     72
375621     70
Name: product_quantity, dtype: int64

In [ ]:
df.groupby(['id_order'])['product_quantity'].sum().sort_values(ascending=False).head(6)

id_order
358747    1081
346221     999
349475     800
349133     555
484334     264
395611     256
Name: product_quantity, dtype: int64

In [ ]:
order_lines.info()

**2. Which are the top 6 orders for the 1 of July of 2017?**

Combining `.assign()` with `.dt.date` will help you extract the date of a datetime column and use this information to filter it. Use the `datetime` module and its `date.fromisoformat()` method as we showed above.

Here is an example how to filter the first of January 2017.

In [ ]:
# Example
df_date = df.assign(date = df['date'].dt.date).copy()
df_date[df_date['date'] == datetime.date.fromisoformat('2017-01-01')]

In [ ]:
df_date = df.assign(date = df['date'].dt.date).copy()
(
df_date[df_date['date'] == datetime.date.fromisoformat('2017-07-01')].groupby(['id_order'])['product_quantity'].sum().sort_values(ascending=False).head(6)
)

In [ ]:
# Example
df_date = df.assign(date = df['date'].dt.date).copy()
(
df_date[df_date['date'] == datetime.date.fromisoformat('2017-07-01')]
                           .groupby(['id_order']).agg({'product_quantity':'sum'})
                           .sort_values(('product_quantity'), ascending=False)
                           .reset_index().head(6)
)                           

,id_order,product_quantity
0,371355,8
1,371217,7
2,371309,7
3,371285,5
4,371178,5
5,371120,5


In [ ]:
(
df
    .assign(month = df['date'].dt.strftime('%m'))
    .query('month == "07"')
    .groupby('id_order')
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)

)

df_new = df.assign(month = df['date'].dt.strftime('%m'))
df_new.head()

**3. Which is the month with the highest number of units sold?** 

Remember to look at the documentation of [`datetime.strftime()`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) 

In [ ]:
df.assign(month = df['date'].dt.strftime('%B')).groupby(['month'])['product_quantity'].sum().sort_values(ascending=False).head(6)

In [ ]:
(
    df.assign(month = df['date'].dt.strftime('%B'))
          .groupby(['month'])
          .agg({'product_quantity':'sum'})
          .sort_values(('product_quantity'), ascending=False)
          .reset_index()
) 

,month,product_quantity
0,January,66415
1,November,46375
2,February,40542
3,December,39094
4,March,29252
5,October,18178
6,July,17923
7,April,15909
8,September,14698
9,May,14392


In the results there is an enormous differences between some months. Why do you think this is happening? Do we only have one year of data? In case you have multiple years, filter for only years that are complete to find out the best month for that year.

Tip: the output of the function `.dt.strftime()` is a string, so if you use it in a condition it will only match with strings!

In [ ]:
new=df.assign(month = df['date'].dt.strftime('%m'))
new=new.assign(year=df['date'].dt.strftime('%y'))
new.groupby("year").agg({"month":"unique"})

,month
year,
17,"[01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12]"
18,"[01, 02, 03]"


In [ ]:
(
df
    .assign(month = df['date'].dt.strftime('%B'),  #df.assign(month = df['date'].dt.strftime('%B, %Y'))
           year = df['date'].dt.strftime('%Y'))
    .query('year == "2017"')
    .groupby('month')
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
)

**4. For each month, find the day of the week with the highest amount of products sold** 

See below an example of the expected output.

In [ ]:
(
    df.assign(day = df['date'].dt.strftime('%B, %A, %Y')) #%D%x  shravanti
          .groupby(['day'])
          .agg({'product_quantity':'sum'})
          .sort_values(('product_quantity'), ascending=False)
          .reset_index()
)

,day,product_quantity
0,"November, Friday, 2017",9717
1,"January, Tuesday, 2018",8403
2,"January, Monday, 2018",8327
3,"November, Thursday, 2017",7727
4,"November, Wednesday, 2017",7137
...,...,...
100,"March, Saturday, 2017",1164
101,"May, Sunday, 2017",1150
102,"June, Saturday, 2017",1083
103,"August, Saturday, 2017",1080


In [ ]:
pd.DataFrame({
    'month':['November','January','July','...'],
    'week_day':['Friday','Tuesday','Monday','...'],
    'product_quantity':[10000, 9800, 9700, 0]
})

In [ ]:
(
df
    .assign(month = df['date'].dt.strftime('%B'),
           year = df['date'].dt.strftime('%Y'), 
           week_day = df['date'].dt.strftime('%A'))
    .query('year == "2017"')
    .groupby(['month','week_day'])
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
    .reset_index()
    .groupby('month')
    .head(10)
)

,month,week_day,product_quantity
0,November,Friday,9717
1,November,Thursday,7727
2,November,Wednesday,7137
3,November,Monday,7109
4,December,Wednesday,7010
...,...,...,...
79,March,Saturday,1164
80,May,Sunday,1150
81,June,Saturday,1083
82,August,Saturday,1080
